In [1]:
import os

def move_to_parent_dir():
    current_directory = os.getcwd()
    parent_directory = os.path.dirname(current_directory)
    os.chdir(parent_directory)
    print("Working directory:", os.getcwd())

move_to_parent_dir()

Working directory: c:\Users\wwden\OneDrive\Documents\GitHub\dsmp-2024-group-19


In [2]:
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
import numpy as np
from tools import get_LOBs

for day in range(125):
    df = get_LOBs(1,day)[0]

    date = str(df.index[0].date())

    print(f"LOB: {date}")
    N = len(df)
    LOB_arr = np.zeros((N,800), dtype = np.int8)
    row_i = 0
    for i, row in df.iterrows():

        bids = row["LOB"][0][1]
        asks = row["LOB"][1][1]

        for price, size in bids:
            LOB_arr[row_i, price - 1] = -size

        for price, size in asks:
            LOB_arr[row_i, price - 1] = size

        row_i += 1



    print("Density:", 1 - np.mean(LOB_arr == 0))
    save_npz(os.path.join("CSR_Data", f"CSR_LOB_{date}.npz"), csr_matrix(LOB_arr, dtype=np.int8))
    with open(os.path.join("CSR_Data", f"Timestamps_{date}.csv"), 'w') as f:
        for timestamp in df.index:
            f.write(str(timestamp.time()) + "\n")


Opening UoB_Set01_2025-01-02LOBs.txt
LOB: 2025-01-02
Density: 0.018302037000311633
Opening UoB_Set01_2025-01-03LOBs.txt
LOB: 2025-01-03
Density: 0.016811435298289212
Opening UoB_Set01_2025-01-06LOBs.txt
LOB: 2025-01-06
Density: 0.018428771405774813
Opening UoB_Set01_2025-01-07LOBs.txt
LOB: 2025-01-07
Density: 0.019879855302015637
Opening UoB_Set01_2025-01-08LOBs.txt
LOB: 2025-01-08
Density: 0.019893356763834302
Opening UoB_Set01_2025-01-09LOBs.txt
LOB: 2025-01-09
Density: 0.019338900459038966
Opening UoB_Set01_2025-01-10LOBs.txt
LOB: 2025-01-10
Density: 0.018965379618937628
Opening UoB_Set01_2025-01-13LOBs.txt
LOB: 2025-01-13
Density: 0.01730369309869162
Opening UoB_Set01_2025-01-14LOBs.txt
LOB: 2025-01-14
Density: 0.019447781569499223
Opening UoB_Set01_2025-01-15LOBs.txt
LOB: 2025-01-15
Density: 0.018337674697835316
Opening UoB_Set01_2025-01-16LOBs.txt
LOB: 2025-01-16
Density: 0.016886458600129828
Opening UoB_Set01_2025-01-17LOBs.txt
LOB: 2025-01-17
Density: 0.01745794313118343
Openin

AssertionError: 

In [3]:
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
import numpy as np
from tools import get_LOBs

df = get_LOBs(1,124)[0]

date = str(df.index[0].date())

print(f"LOB: {date}")
N = len(df)
LOB_arr = np.zeros((N,800), dtype = np.int8)
row_i = 0
for i, row in df.iterrows():

    bids = row["LOB"][0][1]
    asks = row["LOB"][1][1]

    for price, size in bids:
        LOB_arr[row_i, price - 1] = -size

    for price, size in asks:
        LOB_arr[row_i, price - 1] = size

    row_i += 1



print("Density:", 1 - np.mean(LOB_arr == 0))
save_npz(os.path.join("CSR_Data", f"CSR_LOB_{date}.npz"), csr_matrix(LOB_arr, dtype=np.int8))
with open(os.path.join("CSR_Data", f"Timestamps_{date}.csv"), 'w') as f:
    for timestamp in df.index:
        f.write(str(timestamp.time()) + "\n")

Opening UoB_Set01_2025-07-01LOBs.txt
LOB: 2025-07-01
Density: 0.015857565097441406
